<a href="https://colab.research.google.com/github/ArvindRajen/NLP/blob/main/NLP_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score

import json

### Load In Data

#### Data Class

In [ ]:
class Review:
    def __init__(self, category, text):
        self.category = category
        self.text = text    
        
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
    
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_y(self):
        return [x.category for x in self.reviews]

#### Prep Training/Test Data

In [ ]:
train_reviews = []
all_categories = []
for file in os.listdir('./data/training'):
    category = file.strip('train_').split('.')[0]
    all_categories.append(category)
    with open(f'./data/training/{file}') as f:
        for line in f:
            review_json = json.loads(line)
            review = Review(category, review_json['reviewText'])
            train_reviews.append(review)

train_container = ReviewContainer(train_reviews)

In [ ]:
test_reviews = []
for file in os.listdir('./data/test'):
    category = file.strip('test_').split('.')[0]
    with open(f'./data/test/{file}') as f:
        for line in f:
            review_json = json.loads(line)
            review = Review(category, review_json['reviewText'])
            test_reviews.append(review)
            
test_container = ReviewContainer(test_reviews)

#### Train Model (Bag of words)

In [ ]:
from sklearn import svm

corpus = train_container.get_text()
vectorizer = CountVectorizer(binary=True)
train_x = vectorizer.fit_transform(corpus) # training text converted to vector

clf = svm.SVC(kernel='linear')
clf.fit(train_x, train_container.get_y())

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

#### Evaluate Performance (Bag of words)

In [ ]:
# make sure to convert test text to vector form
test_corpus = test_container.get_text()
test_x = vectorizer.transform(test_corpus)

In [ ]:
print("Overall Accuracy:", clf.score(test_x, test_container.get_y()))

y_pred = clf.predict(test_x)

print("f1 scores by category")
print(all_categories)
print(f1_score(test_container.get_y(), y_pred, average=None, labels=all_categories))

Overall Accuracy: 0.6522222222222223
f1 scores by category
['Automotive', 'Beauty', 'Books', 'Clothing', 'Digital_Music', 'Electronics', 'Grocery', 'Patio_Lawn_Garden', 'Pet_Supplies']
[0.46201074 0.79538905 0.82866044 0.71020408 0.71557971 0.5484222
 0.70614035 0.46501129 0.66816143]
